In [59]:
import sys
# # Use these code if you need to install packages
#!{sys.executable} -m pip install schedule
#!{sys.executable} -m pip install twilio

In [60]:
import requests
from bs4 import BeautifulSoup
from twilio.rest import Client
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os

In [61]:
header = {
    'From': 'personal@gmail.com',
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
    'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8'
}

In [62]:
# Using different selector at different platform to scrape price
def get_price(my_df):

    platform = my_df["platform"]
    url = my_df["url"]
    response = requests.get(url, headers=header)
    soup = BeautifulSoup(response.content, 'html.parser')
    try:
        if platform == 'Amazon':
            item_price = "0"
            
        elif platform == 'Gear4':
            item_price = soup.select_one("span.c-val").text
            
        elif platform == 'Thomann':
            item_price = soup.select_one("div.price-wrapper div.price").text
        
        elif platform == 'GuitarCenter':
            item_price = soup.select_one("div.jsx-3452076525.price-condition").text     
            
    except IndexError:
            item_price = "999999999"

    item_price = item_price.replace(',', '').replace('$', '').replace('£', '').replace('.00', '')
    return float(item_price)

In [63]:
# Read master data about item, url, platform and url into data frame
path = os.getcwd()
my_df = pd.read_csv(f'{path}/master_Data.csv')
my_df.head()

,platform,nickname,product_name,target_price,url
0,Amazon,Fender Player Plus,Fender Player?Telecaster?Electric Guitar - Map...,1500.0,https://www.amazon.com/Fender-Player-Telecaste...
1,GuitarCenter,Fender Player Plus,Fender Player Plus Telecaster Maple Fingerboar...,900.4,https://www.guitarcenter.com/Fender/Player-Plu...
2,Thomann,Fender Player Plus,Fender Player Plus Strat Aged CAR,750.0,https://www.thomannmusic.com/fender_player_plu...
3,Gear4,Fender Player Plus,"Fender Player Plus Telecaster MN, Aged Candy A...",750.0,https://www.gear4music.com/us/en/Guitar-and-Ba...
4,Amazon,PRS Santana Retro,PRS McCarty Singlecut 594 Figured Top - Black ...,3000.0,https://www.amazon.com/PRS-McCarty-Singlecut-5...


In [64]:
# Apply get_price function to the data frame, and record the scrape time
my_df["updated_price"] = my_df.apply(get_price,axis = 1)
my_df["updated_time"] = datetime.now().strftime('%Y/%m/%d %H:%M:%S')
my_df.head()

,platform,nickname,product_name,target_price,url,updated_price,updated_time
0,Amazon,Fender Player Plus,Fender Player?Telecaster?Electric Guitar - Map...,1500.0,https://www.amazon.com/Fender-Player-Telecaste...,0.00,2022/12/13 20:30:06
1,GuitarCenter,Fender Player Plus,Fender Player Plus Telecaster Maple Fingerboar...,900.4,https://www.guitarcenter.com/Fender/Player-Plu...,1099.99,2022/12/13 20:30:06
2,Thomann,Fender Player Plus,Fender Player Plus Strat Aged CAR,750.0,https://www.thomannmusic.com/fender_player_plu...,839.00,2022/12/13 20:30:06
3,Gear4,Fender Player Plus,"Fender Player Plus Telecaster MN, Aged Candy A...",750.0,https://www.gear4music.com/us/en/Guitar-and-Ba...,906.00,2022/12/13 20:30:06
4,Amazon,PRS Santana Retro,PRS McCarty Singlecut 594 Figured Top - Black ...,3000.0,https://www.amazon.com/PRS-McCarty-Singlecut-5...,0.00,2022/12/13 20:30:06


In [65]:
# Put latest data underneath the exsiting data if they exist, and read in all of them
try:
    old_df = pd.read_csv(f'{path}/updated_product_price.csv')
    df_list = [old_df,my_df]
    merged_df = pd.concat(df_list, axis = 0)
except FileNotFoundError:
    merged_df = my_df
merged_df.to_csv(f'{path}/updated_product_price.csv',index = False)
merged_df = pd.read_csv(f'{path}/updated_product_price.csv')

In [66]:
# Check the updated data with 'updated_time' and 'updated price'
# Sort the date by 'nickname' and 'platform'
merged_df[['nickname','platform','target_price', 'updated_price','updated_time']].sort_values(by=['nickname','platform']).head()

,nickname,platform,target_price,updated_price,updated_time
0,Fender Player Plus,Amazon,1500.0,0.0,2022/12/13 20:18:53
8,Fender Player Plus,Amazon,1500.0,0.0,2022/12/13 20:20:33
16,Fender Player Plus,Amazon,1500.0,0.0,2022/12/13 20:30:06
3,Fender Player Plus,Gear4,750.0,906.0,2022/12/13 20:18:53
11,Fender Player Plus,Gear4,750.0,906.0,2022/12/13 20:20:33


In [67]:
# Visualize above updated data with histgrams
g=sns.catplot(x="platform", y="updated_price", hue="updated_time", col="nickname", data=merged_df, kind="bar", palette='Blues')
g.fig.set_size_inches(15, 8) # Set the chart size
g.fig.subplots_adjust(top=0.9) # Set the top margin of chart
g.fig.suptitle('Product Price Tracking', size=16) # Set the chart title and font size


for ax in g.axes.ravel():
    for c in ax.containers:
        labels = [f'{(v.get_height()):.1f}' for v in c]
        ax.bar_label(c, labels=labels, label_type='edge', size=6)
    ax.margins(y=0.2) # Add values to each bar and set the font style

In [68]:
#Draw lineplot for each product-platform to track the price changing
sns.set(style = "white")
fmri = merged_df
sns.relplot(x = "updated_time", y = "updated_price",col = 'nickname',kind = 'line', data = fmri)


In [69]:
#Draw heatmap for each product to compare the price at different platform and different time
for item in merged_df["nickname"].unique():
    item_history = merged_df[merged_df["nickname"] == item]
#     item_history.loc[:,"updated_time"] = pd.to_datetime(item_history.loc[:,"updated_time"])
    pt = item_history.pivot_table(index = 'platform',columns = 'updated_time',values = 'updated_price',aggfunc = np.average)
    f,(ax) = plt.subplots(figsize=(12,4))
    sns.heatmap(pt,annot = True, ax = ax, fmt = 'g',annot_kws={'size':15},cmap='YlGnBu',linewidths=1)
    ax.set_xlabel('Time')
    ax.set_ylabel('Product')
    ax.set_title(item)


In [70]:
# Collect items if their price drop below the target
price_drop_list_url = []
price_drop_message = []

# Check the latest scrape result to find items price drop below the target
for row in my_df.iterrows():

    product_url = row[1]['url']
    target_price = row[1]['target_price']
    nickname = row[1]['nickname']
    platform = row[1]['platform']
    updated_price = row[1]['updated_price']

    if updated_price < target_price:
        change_percentage = round((target_price - updated_price) * 100 / target_price) #checking if what percentage of price changed
        # Format a message with shopping url and latest price if the price drop below the target
        item_message = '{}'.format(nickname) + ' from {}'.format(platform) + ' is now {}'.format(updated_price) + ', {}'.format(change_percentage) + '% lower than your target price {}'.format(target_price) +"."+ "\n" + 'Click here to purchase ' + format(product_url)
        price_drop_list_url.append(row[1]['url'])
        price_drop_message.append(item_message)
        print(item_message)

Fender Player Plus from Amazon is now 0.0, 100% lower than your target price 1500.0.
Click here to purchase https://www.amazon.com/Fender-Player-Telecaster-Electric-Guitar/dp/B07CXJSFTH/
PRS Santana Retro from Amazon is now 0.0, 100% lower than your target price 3000.0.
Click here to purchase https://www.amazon.com/PRS-McCarty-Singlecut-594-Figured/dp/B01N319MXL


In [71]:
# Format the header of SMS
messege = "There is {}".format(len(price_drop_list_url)) + " products lower than target prices."#sms content

In [72]:
if len(price_drop_list_url) == 0: #if no price changes found 
    print('No Price drop found')
    
else:
    # Put every drop message behind the header and sent
    for item_message in price_drop_message:
        messege = messege + "\n" + item_message

    account_sid = 'ACdec2e0c430fcf690040b958411d5dbb2'
    auth_token = '00fe5a955299874ab5ef65d0879f64c0'

    client = Client(account_sid, auth_token)
    message = client.messages.create(
        from_ ='+1000000000',
        body = messege,
        to ='+100000000'
    )
    print('Price drop found')

Price drop found
